In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

mpl.rc('font', family='Malgun Gothic')

path = "../"

In [16]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
df = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+'test.csv')

In [13]:
pd.set_option("display.max_columns", 30)

In [14]:
df

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,20.0,10.0,38600,220.0,"China, People's Republic Of",0.14,3.77,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,1.114595
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,20.0,10.0,28100,220.0,Singapore,-3.16,-6.72,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,2.841334
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,20.0,10.0,96400,300.0,Liberia,0.00,0.00,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,4.593376
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,10.0,0.0,700,70.0,Nauru,4.34,-7.31,22.1,4.693735,7,43.02,43.15,41.11,1169.853455,0.000069,-inf
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,10.0,10.0,19300,180.0,Panama,1.92,2.31,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,4.564666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,20.0,10.0,51300,230.0,Liberia,NaN,NaN,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,4.187379
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,0.0,0.0,500,60.0,Japan,-0.06,0.87,17.1,1.028558,11,105.37,109.34,104.69,1955.103846,0.000552,-inf
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,20.0,10.0,25100,180.0,Cyprus,-0.63,3.36,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,-0.002503
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,10.0,0.0,750,80.0,Japan,3.74,-2.44,10.8,3.055715,19,49.75,50.08,47.02,1191.353331,0.000069,-inf


In [11]:
df.drop(columns = ['SHIPMANAGER'], inplace=True)

test.drop(columns = ['SHIPMANAGER'], inplace=True)

In [12]:
df['CI_HOUR'] = np.log(df['CI_HOUR'])

C:\Users\dohyeong\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
df.loc[df['DIST']==0, 'd_zero'] = 1
df.loc[df['DIST']!=0, 'd_zero'] = 0

In [20]:
ohe = OneHotEncoder(handle_unknown='ignore', min_frequency=5, sparse=False, drop='first')

In [21]:
ohe.fit(df[['ARI_PO', 'ARI_CO', 'SHIP_TYPE_CATEGORY']])

C:\Users\dohyeong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(drop='first', handle_unknown='ignore', min_frequency=5,
              sparse=False, sparse_output=False)

In [31]:
encoded = ohe.transform(df[['ARI_PO', 'ARI_CO', 'SHIP_TYPE_CATEGORY']])

In [32]:
encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
df.drop(columns = ['BREADTH', "GT", "BRENT"], inplace=True)
test.drop(columns=['BREADTH', "GT", 'BRENT',], inplace=True)

In [25]:
df

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,LENGTH,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR,d_zero
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,28,73100,20.0,10.0,220.0,"China, People's Republic Of",0.14,3.77,15.9,2.730798,12,42.01,40.96,1407.668330,0.001660,1.114595,0.0
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,15,37900,20.0,10.0,220.0,Singapore,-3.16,-6.72,24.5,4.289058,10,67.53,59.34,2089.046774,0.001614,2.841334,0.0
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,7,115000,20.0,10.0,300.0,Liberia,0.00,0.00,9.4,0.000000,14,65.30,56.94,603.193047,0.001743,4.593376,0.0
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,33,1490,10.0,0.0,70.0,Nauru,4.34,-7.31,22.1,4.693735,7,43.02,41.11,1169.853455,0.000069,-inf,1.0
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,10,27600,10.0,10.0,180.0,Panama,1.92,2.31,22.8,2.345875,14,90.45,88.11,1107.944894,0.000197,4.564666,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,13,93200,20.0,10.0,230.0,Liberia,NaN,NaN,NaN,NaN,6,61.25,55.70,1333.609109,0.000360,4.187379,0.0
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,9,1280,0.0,0.0,60.0,Japan,-0.06,0.87,17.1,1.028558,11,105.37,104.69,1955.103846,0.000552,-inf,1.0
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,6,25000,20.0,10.0,180.0,Cyprus,-0.63,3.36,31.7,2.557156,15,97.73,95.78,1601.291086,0.002615,-0.002503,0.0
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,8,2400,10.0,0.0,80.0,Japan,3.74,-2.44,10.8,3.055715,19,49.75,47.02,1191.353331,0.000069,-inf,1.0


In [33]:
encoded = pd.DataFrame(encoded, columns = ohe.get_feature_names_out())

In [34]:
encoded

,ARI_PO_AZU6,ARI_PO_BAZ5,ARI_PO_BGD2,ARI_PO_BGX4,ARI_PO_CEI5,ARI_PO_CSP6,ARI_PO_CXL1,ARI_PO_DEJ2,ARI_PO_DIN2,ARI_PO_DMD4,ARI_PO_EFG4,ARI_PO_EIA2,ARI_PO_EKP8,ARI_PO_EUC8,ARI_PO_EVL6,...,ARI_CO_PE,ARI_CO_PH,ARI_CO_QA,ARI_CO_RU,ARI_CO_SG,ARI_CO_TT,ARI_CO_TW,ARI_CO_UA,ARI_CO_US,ARI_CO_VE,ARI_CO_VN,ARI_CO_ZA,SHIP_TYPE_CATEGORY_Cargo,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
367438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
367439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
target_df = df['CI_HOUR']
df.drop(columns = 'CI_HOUR', inplace=True)


In [38]:
ata = pd.to_datetime(df['ATA'])

In [40]:
ata = pd.DataFrame(ata)

In [42]:
ata['year'] = ata['ATA'].dt.year
ata['month'] = ata['ATA'].dt.month

In [44]:
ata['dmonth'] = (ata['year']-2014)*12 + ata['month']

In [46]:
ata['dayofweek'] = ata['ATA'].dt.dayofweek

In [47]:
ata

,ATA,year,month,dmonth,dayofweek
0,2020-10-15 04:03:00,2020,10,82,3
1,2019-09-17 02:55:00,2019,9,69,1
2,2019-02-23 06:43:00,2019,2,62,5
3,2020-09-18 22:06:00,2020,9,81,4
4,2022-08-13 12:57:00,2022,8,104,5
...,...,...,...,...,...
367436,2017-11-11 22:23:00,2017,11,47,5
367437,2022-04-29 02:58:00,2022,4,100,4
367438,2022-07-14 07:58:00,2022,7,103,3
367439,2020-12-22 10:07:00,2020,12,84,1


In [36]:
df= pd.concat([df, encoded], axis=1)

In [37]:
df

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,LENGTH,FLAG,U_WIND,V_WIND,...,ARI_CO_PE,ARI_CO_PH,ARI_CO_QA,ARI_CO_RU,ARI_CO_SG,ARI_CO_TT,ARI_CO_TW,ARI_CO_UA,ARI_CO_US,ARI_CO_VE,ARI_CO_VN,ARI_CO_ZA,SHIP_TYPE_CATEGORY_Cargo,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,28,73100,20.0,10.0,220.0,"China, People's Republic Of",0.14,3.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,15,37900,20.0,10.0,220.0,Singapore,-3.16,-6.72,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,7,115000,20.0,10.0,300.0,Liberia,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,33,1490,10.0,0.0,70.0,Nauru,4.34,-7.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,10,27600,10.0,10.0,180.0,Panama,1.92,2.31,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,13,93200,20.0,10.0,230.0,Liberia,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,9,1280,0.0,0.0,60.0,Japan,-0.06,0.87,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,6,25000,20.0,10.0,180.0,Cyprus,-0.63,3.36,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,8,2400,10.0,0.0,80.0,Japan,3.74,-2.44,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
